In [1]:
import pandas as pd 
import numpy as np 
import xlrd 
import sklearn

In [2]:
table_specie = pd.read_excel('../data/raw/LCMS_data_copy.xlsx',0)
table_total= pd.read_excel('../data/raw/LCMS_data_copy.xlsx',1)
lip_name_table = pd.read_excel('../data/raw/LCMS_data_copy.xlsx',4)

In [3]:
lip_name_table

,Original name,Updated name,Class,BioGrouping,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Abreviation,old abbreviation,Lipid class
0,Sph(d16:1) Results,Sph(d16:1),Sph,1,NaN,NaN,NaN,NaN,FFA,FFA,free fatty acids
1,Sph(d18:1) Results,Sph(d18:1),Sph,1,NaN,NaN,NaN,NaN,Cer(d18:0/xx:xx),dhCer,dihydroceramide
2,Sph(d18:2) Results,Sph(d18:2),Sph,1,NaN,NaN,NaN,NaN,Cer(d18:1/xx:x),Cer,ceramide
3,S1P(d16:1) Results,S1P(d16:1),S1P,2,NaN,NaN,NaN,NaN,Hex1Cer,MHC,monohexosylceramide
4,S1P(d18:0) Results,S1P(d18:0),S1P,2,NaN,NaN,NaN,NaN,Hex2Cer,DHC,dihexosylceramide
...,...,...,...,...,...,...,...,...,...,...,...
795,LPC 20:4_OH Results,LPC(20:4) [+OH],OxSpecies,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
796,LPC 22:6_OH Results,LPC(22:6) [+OH],OxSpecies,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
797,PC 34:2_OH Results,PC(34:2) [+OH],OxSpecies,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
798,PC 36:4(a\b)_OH Results,PC(36:4) [+OH],OxSpecies,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
feature_list_total = (table_total.iloc[:,2:].columns.values.tolist())[1:] # id and features name
feature_list_specie = (table_specie.iloc[:,2:].columns.values.tolist())[1:] # id and features name 
name_group_link = lip_name_table[['Updated name','Class','BioGrouping']]

In [5]:
len(feature_list_specie)

774

In [6]:
group_name_list = [name_group_link.loc[name_group_link['Updated name']==item]['Class'].values[0] for item in feature_list_specie]# get group name for each specie

In [7]:
len(set(group_name_list)) # nunmber of total groups

41

In [8]:
group_name_dict = {}
for i,item in enumerate(set(group_name_list)):
    group_name_dict[item] = i
group_name_dict

{'LPE(P)': 0,
 'TG(SIM)': 1,
 'Ubiquinone': 2,
 'PC': 3,
 'PI': 4,
 'OxSpecies': 5,
 'LPE': 6,
 'TG(O) [NL]': 7,
 'Cer': 8,
 'TG(O)': 9,
 'THC': 10,
 'dhCer': 11,
 'PS': 12,
 'LPI': 13,
 'LPC(O)': 14,
 'DG': 15,
 'PC(P)': 16,
 'PC(O)': 17,
 'DHC': 18,
 'GM1': 19,
 'PE(O)': 20,
 'PIP1': 21,
 'Sph': 22,
 'Sulfatide': 23,
 'S1P': 24,
 'LPC(P)': 25,
 'PE': 26,
 'PE(P)': 27,
 'C1P': 28,
 'FFA': 29,
 'GM3': 30,
 'CE': 31,
 'Acylcarnitine': 32,
 'TG [NL]': 33,
 'SM': 34,
 'DE': 35,
 'MHC': 36,
 'COH': 37,
 'LPC': 38,
 'PA': 39,
 'PG': 40}

In [9]:
def id_process(id):

    if id[-1] == 'C':
        # pattern 'num + C' 
        return id[:-2]
    elif id[1] == '-':
        # pattern 'P-num' 
        return id[2:]
    else:# pattern 'P num'
        return id[1:]
table_specie.iloc[:,2].apply(id_process)

0      1001
1      1010
2      1019
3      1028
4      1037
       ... 
802    2395
803    2412
804    2429
805    2456
806    2493
Name: Patient ID, Length: 807, dtype: object

In [11]:
table_specie_new = pd.DataFrame()
table_specie_new['Patient_ID'] = table_specie.iloc[:,2].apply(id_process)

for lip_name in feature_list_specie:
    df = table_specie[lip_name]
    # normalized_df = (df-df.min())/(df.max()-df.min())
    # normalized_df = (df-df.mean())/df.std()
    normalized_df = df

    group_name = name_group_link.loc[name_group_link['Updated name']==lip_name]['Class'].values[0]
    group_name_idx = str(group_name_dict[group_name])
    table_specie_new['lip_' + lip_name + '_' +group_name + '_' + group_name_idx] = normalized_df.fillna(0)
table_specie_new    

,Patient_ID,lip_Sph(d16:1)_Sph_22,lip_Sph(d18:1)_Sph_22,lip_Sph(d18:2)_Sph_22,lip_S1P(d16:1)_S1P_24,lip_S1P(d18:0)_S1P_24,lip_S1P(d18:1)_S1P_24,lip_S1P(d18:2)_S1P_24,lip_Cer(d18:0/16:0)_dhCer_11,lip_Cer(d18:0/18:0)_dhCer_11,...,lip_TG(O-54:4) [NL-18:2]_TG(O) [NL]_7,lip_Ubiquinone_Ubiquinone_2,lip_CE(18:2) [+OH]_OxSpecies_5,lip_CE(20:4) [+OH]_OxSpecies_5,lip_CE(22:6) [+OH]_OxSpecies_5,lip_LPC(18:2) [+OH]_OxSpecies_5,lip_LPC(20:4) [+OH]_OxSpecies_5,lip_LPC(22:6) [+OH]_OxSpecies_5,lip_PC(34:2) [+OH]_OxSpecies_5,lip_PC(36:4) [+OH]_OxSpecies_5
0,1001,21.516011,41.558599,15.803974,147.585884,96.676645,854.358849,258.873480,53.784848,17.029827,...,12.143672,26238.373306,2162.300397,879.334303,343.006547,3.324332,4.448883,1.366231,5.666700,114.079475
1,1010,19.511903,56.443014,17.399277,106.137889,87.745079,678.969623,161.695552,40.286656,9.663929,...,8.983172,14670.455442,29634.697976,3011.962878,6038.941206,466.317367,292.434780,49.383246,303.235634,133.934894
2,1019,18.847631,59.554417,14.565941,124.046526,79.788243,482.287200,149.330196,33.874688,6.216366,...,22.281400,15067.552500,3317.642636,655.477471,468.822096,6.071270,1.458804,1.128331,11.766817,74.036710
3,1028,15.571034,50.301966,14.394042,102.431581,87.123449,664.550108,180.301240,17.219216,4.152714,...,6.336801,8490.907446,38886.612154,3842.737616,7945.785177,231.014791,150.146655,32.983950,186.307062,83.304119
4,1037,16.168717,53.583285,16.071080,137.264920,109.802111,640.291234,173.079568,33.804700,9.256049,...,7.553749,16758.169195,1772.650722,642.194812,326.006643,3.170387,3.073136,1.064900,19.644677,109.651570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,2395,18.452590,38.506929,13.527684,80.141784,38.090208,259.640246,85.917754,30.189646,9.541595,...,10.778533,10469.494747,2004.347209,646.130715,919.327769,4.570760,1.593339,0.851453,10.224673,84.660209
803,2412,23.913580,60.415481,18.737205,154.046223,87.509622,656.469560,173.581730,54.716523,15.758825,...,7.495488,24505.334959,6054.799190,1195.696538,433.146587,3.207587,1.060176,0.857117,10.376018,74.201055
804,2429,18.275613,47.307042,15.855248,152.410484,59.381981,606.130140,200.142295,27.712801,10.147275,...,7.322259,26624.313519,2304.828100,981.541775,384.457860,4.052621,2.235764,1.028642,13.681009,72.368560
805,2456,19.912020,57.082641,13.069190,117.242815,73.381760,515.519366,160.400268,36.174652,13.999316,...,12.599320,31194.340154,3115.425628,959.126592,352.485535,3.541962,1.726280,0.769208,6.001037,135.854916


In [20]:
# binarize the lip feature: for each feature ,
#  if val > median, set feature_high = 1,feature_low = 0
#  else,   set feature_high = 0,feature_high = 1

# table_specie_new_bi = pd.DataFrame()
# table_specie_new_bi['Patient_ID'] = table_specie_new['Patient_ID']
# feature_list = list(table_specie_new)


# for lip_name in feature_list[1:]:
#     df = table_specie_new[lip_name]
#     table_specie_new_bi[lip_name + '_high'] = (df>=df.median())*1
#     table_specie_new_bi[lip_name + '_low'] = (df<df.median())*1


# table_specie_new_bi

,Patient_ID,lip_Sph(d16:1)_high,lip_Sph(d16:1)_low,lip_Sph(d18:1)_high,lip_Sph(d18:1)_low,lip_Sph(d18:2)_high,lip_Sph(d18:2)_low,lip_S1P(d16:1)_high,lip_S1P(d16:1)_low,lip_S1P(d18:0)_high,...,lip_LPC(18:2) [+OH]_high,lip_LPC(18:2) [+OH]_low,lip_LPC(20:4) [+OH]_high,lip_LPC(20:4) [+OH]_low,lip_LPC(22:6) [+OH]_high,lip_LPC(22:6) [+OH]_low,lip_PC(34:2) [+OH]_high,lip_PC(34:2) [+OH]_low,lip_PC(36:4) [+OH]_high,lip_PC(36:4) [+OH]_low
0,1001,1,0,0,1,0,1,1,0,1,...,0,1,1,0,1,0,0,1,1,0
1,1010,1,0,1,0,1,0,0,1,1,...,1,0,1,0,1,0,1,0,1,0
2,1019,1,0,1,0,0,1,1,0,1,...,1,0,1,0,1,0,1,0,1,0
3,1028,0,1,0,1,0,1,0,1,1,...,1,0,1,0,1,0,1,0,1,0
4,1037,0,1,0,1,0,1,1,0,1,...,0,1,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,2395,1,0,0,1,0,1,0,1,0,...,1,0,1,0,0,1,1,0,1,0
803,2412,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,1,0,1,0
804,2429,1,0,0,1,0,1,1,0,0,...,1,0,1,0,1,0,1,0,1,0
805,2456,1,0,1,0,0,1,1,0,1,...,0,1,1,0,0,1,0,1,1,0


In [12]:
table_specie_new.to_csv('../data/processed/process_lip_specie_raw.csv',index=False)
# table_specie_new_bi.to_csv('../processed_data/process_lip_specie_binary.csv',index=False)

In [49]:
idset = set(table_specie_new['Patient_ID'])

In [50]:
len(idset) # 2two overlaping id 

805